In [29]:
import random
import torch
import numpy as np
from tqdm.auto import trange
from tqdm import tqdm

from scipy.stats.qmc import Halton

from scipy.optimize import minimize

from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement
from botorch.test_functions import Hartmann

from botorch.models.transforms import Standardize
from botorch.models.transforms.input import Normalize



from matplotlib import pyplot as plt
plt.style.use('ggplot')

SEEDS = np.loadtxt('seeds.txt', delimiter=',', dtype=int)

In [30]:
def set_seeds(seed=42):
    """set all library random seeds"""
    seed = int(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def random_sample(num_points, dims, seed=42):
    """generate random points in the domain"""
    set_seeds(seed)
    return torch.rand(num_points, dims, dtype=torch.double)

def execute_campaign(n_campaigns=10, campaign_budget=30, n_init_samples=5, raw_samples=250, num_restarts=20):

    data = np.zeros((n_campaigns, campaign_budget))
    campaign_budget = campaign_budget - n_init_samples

    HART = Hartmann(dim=6, bounds=[(0,1)]*6 ,negate=True) # objective function - call with X, returns y values (like an experiment)
    X_ = random_sample(n_init_samples, 6) #  6 is num of dims 
    y_ = torch.tensor([HART(x) for x in X_])[:,None]

    for campaign in trange(n_campaigns, desc=f'Running RS={raw_samples}, NR={num_restarts}'):
        set_seeds(SEEDS[campaign])
        X = X_.clone()
        y = y_.clone()

        for trial in range(campaign_budget):

            gp = SingleTaskGP(
                train_X = X,
                train_Y = y,
                input_transform=Normalize(d=X.shape[-1]), # normalize X values [0,1]
                outcome_transform=Standardize(m=y.shape[-1]) # standardize y values [mean of 0, std of 1]
            )

            # fit the model by maximizing the log marginal likelihood
            mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
            mll = fit_gpytorch_model(mll)

            bounds = torch.tensor([[0.0]*X.shape[-1], [1.0]*X.shape[-1]])

            X_new, acq_value = optimize_acqf( # acq value is whatever came out of acqf (y value of acqf)
                acq_function= UpperConfidenceBound(gp, beta=2),  # beta= 2 std from mean
                bounds=bounds, # where to search for X values
                q=1, # how many new points to generate # one new experiment to do
                num_restarts = num_restarts, # how many times to restart the optimizer
                raw_samples = raw_samples # how many initial points to sample acqf space from
            )

            X = torch.cat([X, X_new])
            y = torch.cat([y, HART(X=X_new)[:,None]]) # new exp observation is HART(X_new)

        data[campaign,:] = y.flatten()
    
    return data

def generate_points(vis=False):
    points = Halton(2, optimization='lloyd', seed=3).random(150)
    points = points * np.array([200, 2048]) + 1
    points = points.astype(np.int32)
    return points
    if vis:
        plt.scatter(points[:,0], points[:,1])
        plt.xlabel('num_restarts')
        plt.ylabel('raw_samples')
        plt.show()

In [31]:
sampling_points = generate_points(vis=False)

for i, param in enumerate(sampling_points):
    num_restarts, raw_samples = param
    num_restarts = int(num_restarts)
    raw_samples = int(raw_samples)
    ensemble = execute_campaign(
        n_campaigns=3,
        campaign_budget=15,
        n_init_samples=5,
        num_restarts=num_restarts, 
        raw_samples=raw_samples
    )
    # save ensemble to csv
    np.savetxt(f'ensemble_res/ensemble_{num_restarts}_{raw_samples}.csv', ensemble, delimiter=',')
    if i ==1:
        break

Running RS=966, NR=9: 100%|██████████| 3/3 [00:04<00:00,  1.63s/it]
